In [1]:
import os
import numpy as np
import torch
import torch_geometric.data

import meld_graph
import meld_graph.training
import meld_graph.models
import meld_graph.experiment
import meld_graph.dataset
from meld_graph.paths import load_config
from meld_graph.paths import EXPERIMENT_PATH
from meld_graph.dataset import GraphDataset, Oversampler

Setting MELD_DATA_PATH to /rds/project/kw350/rds-kw350-meld/meld_data/Data
Setting BASE_PATH to /rds/project/kw350/rds-kw350-meld/meld_data/Data
Setting EXPERIMENT_PATH to /rds/project/kw350/rds-kw350-meld/experiments/co-ripa1/
Setting FS_SUBJECTS_PATH to 
Setting EXPERIMENT_PATH to /rds/project/kw350/rds-kw350-meld/experiments_graph/co-ripa1


In [2]:
# create experiment
config_file = '/rds/user/co-ripa1/hpc-work/scripts/meld_classifier_GDL/scripts/config_files/base_config_test_histology.py'
config = load_config(config_file)
exp = meld_graph.experiment.Experiment(config.network_parameters, config.data_parameters)


Saving parameter files to /rds/project/kw350/rds-kw350-meld/experiments_graph/co-ripa1/23-08-16_MATH_histology_head_noseg_lesionfeat_oversampler/fold_00
Initialised Experiment 23-08-16_MATH_histology_head_noseg_lesionfeat_oversampler


In [42]:
import importlib
importlib.reload(meld_graph.dataset)
importlib.reload(meld_graph.data_preprocessing)
importlib.reload(meld_graph.training)

<module 'meld_graph.training' from '/rds/user/co-ripa1/hpc-work/scripts/meld_classifier_GDL/meld_graph/training.py'>

In [3]:
# create the trainer the model
trainer = meld_graph.training.Trainer(exp)
trainer.experiment.load_model()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
trainer.experiment.model.to(device)
# trainer.train()


Creating model


In [4]:
train_dset = GraphDataset.from_experiment(trainer.experiment, mode='train')
print(train_dset)


Getting train val test split
total number of subjects: 335
total number of subjects after restricting to subjects from MELD_dataset_V6_8020.csv: 326
total number of subjects: 950
total number of subjects after restricting to subjects from MELD_dataset_V6_8020.csv: 912
total number after filtering by scanner ['3T', '15T'], features, lesional_only True: 912
full_feature_list: ['.combat.on_lh.curv.sm5.mgh', '.combat.on_lh.gm_FLAIR_0.25.sm10.mgh', '.combat.on_lh.gm_FLAIR_0.5.sm10.mgh', '.combat.on_lh.gm_FLAIR_0.75.sm10.mgh', '.combat.on_lh.gm_FLAIR_0.sm10.mgh', '.combat.on_lh.pial.K_filtered.sm20.mgh', '.combat.on_lh.sulc.sm5.mgh', '.combat.on_lh.thickness.sm10.mgh', '.combat.on_lh.w-g.pct.sm10.mgh', '.combat.on_lh.wm_FLAIR_0.5.sm10.mgh', '.combat.on_lh.wm_FLAIR_1.sm10.mgh', '.inter_z.asym.intra_z.combat.on_lh.curv.sm5.mgh', '.inter_z.asym.intra_z.combat.on_lh.gm_FLAIR_0.25.sm10.mgh', '.inter_z.asym.intra_z.combat.on_lh.gm_FLAIR_0.5.sm10.mgh', '.inter_z.asym.intra_z.combat.on_lh.gm_FLAIR_0

GraphDataset(416)


In [6]:
val_dset = GraphDataset.from_experiment(trainer.experiment, mode='val')
print(val_dset)

Loading and preprocessing val data
Z-scoring data for MELD_H4_3T_FCD_0011
Z-scoring data for MELD_H4_15T_FCD_0022
Z-scoring data for MELD_H4_15T_FCD_0011
Z-scoring data for MELD_H4_15T_C_0017
Z-scoring data for MELD_H4_3T_FCD_0021
Z-scoring data for MELD_H4_3T_FCD_0033
Z-scoring data for MELD_H4_15T_C_0006
Z-scoring data for MELD_H4_15T_C_0003
Z-scoring data for MELD_H4_3T_FCD_0042


GraphDataset(18)


In [12]:
importlib.reload(meld_graph.dataset)

<module 'meld_graph.dataset' from '/rds/user/co-ripa1/hpc-work/scripts/meld_classifier_GDL/meld_graph/dataset.py'>

In [25]:
len(train_dset)

416

In [22]:
sampler = Oversampler(train_dset)

/rds/user/co-ripa1/hpc-work/scripts/meld_classifier_GDL/meld_graph/dataset.py:341: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self._histo_idxs = np.array([np.array(histo_0_idxs), np.array(histo_1_idxs), np.array(histo_2_idxs), np.array(histo_3_idxs)])


In [27]:
len(sampler.get_sampling_idxs())

162

In [28]:

sampler = Oversampler(train_dset)
# shuffle = trainer.params['shuffle_each_epoch']
shuffle = False
batch_size = trainer.params['batch_size']
optimiser = torch.optim.SGD(trainer.experiment.model.parameters(), **trainer.params['optimiser_parameters'])
optimiser = optimiser

train_data_loader = torch_geometric.loader.DataLoader(
             train_dset, sampler=sampler, 
             shuffle=shuffle,
             batch_size= batch_size,
             num_workers=0, persistent_workers=False, prefetch_factor=2
             )

In [33]:
train_data_loader.dataset

GraphDataset(416)

In [35]:
len(sampler.get_sampling_idxs())

162

In [18]:
cur_scores = trainer.train_epoch(train_data_loader, optimiser)

KeyboardInterrupt: 

In [ ]:
cur_scores

{'cross_entropy': 0.7883511781692505,
 'dice': 0.9784323046915233,
 'distance_regression': 0.28809136897325516,
 'lesion_classification': 0.6396960318088531,
 'histology_classification': 1.3424707651138306,
 'ds6_cross_entropy': 0.008665163069963455,
 'ds5_cross_entropy': 0.02071181684732437,
 'ds4_cross_entropy': 0.041922105476260185,
 'ds3_cross_entropy': 0.06115961913019419,
 'ds2_cross_entropy': 0.1806270331144333,
 'ds1_cross_entropy': 0.34350793808698654,
 'ds6_dice': 0.014627989630064462,
 'ds5_dice': 0.03026165576011408,
 'ds4_dice': 0.05998166825156659,
 'ds3_dice': 0.11645015981048346,
 'ds2_dice': 0.22800071304664016,
 'ds1_dice': 0.4208209812641144,
 'ds6_distance_regression': 0.004387676890473813,
 'ds5_distance_regression': 0.011066023958846927,
 'ds4_distance_regression': 0.02937454916536808,
 'ds3_distance_regression': 0.04111327417194843,
 'ds2_distance_regression': 0.15398433804512024,
 'ds1_distance_regression': 0.27838488668203354,
 'ds6_lesion_classification': nan,

In [4]:
trainer.train(wandb_logging=False)

Model already exists. Specify force=True to force reloading and initialisation
Creating model
Getting train val test split
total number of subjects: 624
total number of subjects after restricting to subjects from MELD_dataset_V6_8020.csv: 609
total number of subjects: 950
total number of subjects after restricting to subjects from MELD_dataset_V6_8020.csv: 912
total number after filtering by scanner ['3T', '15T'], features, lesional_only True: 912
full_feature_list: ['.combat.on_lh.curv.sm5.mgh', '.combat.on_lh.gm_FLAIR_0.25.sm10.mgh', '.combat.on_lh.gm_FLAIR_0.5.sm10.mgh', '.combat.on_lh.gm_FLAIR_0.75.sm10.mgh', '.combat.on_lh.gm_FLAIR_0.sm10.mgh', '.combat.on_lh.pial.K_filtered.sm20.mgh', '.combat.on_lh.sulc.sm5.mgh', '.combat.on_lh.thickness.sm10.mgh', '.combat.on_lh.w-g.pct.sm10.mgh', '.combat.on_lh.wm_FLAIR_0.5.sm10.mgh', '.combat.on_lh.wm_FLAIR_1.sm10.mgh', '.inter_z.asym.intra_z.combat.on_lh.curv.sm5.mgh', '.inter_z.asym.intra_z.combat.on_lh.gm_FLAIR_0.25.sm10.mgh', '.inter_z.as

KeyboardInterrupt: 